In [ ]:
%load_ext autoreload
%autoreload 2
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-864c07c4-8eeb-5b23-8d57-eaeb942a9a0f"
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml
from hydra.utils import instantiate
from PIL import Image
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader, Dataset

device = "cuda:0"

In [ ]:
os.chdir("/allen/aics/modeling/ritvik/projects/benchmarking_representations/")

In [ ]:
from cyto_dl.models.utils.mlflow import load_model_from_checkpoint

from br.data.get_datamodules import get_data
from br.models.load_models import load_model_from_path
from br.models.save_embeddings import get_pc_loss


def get_data_and_models(dataset_name, batch_size, results_path, debug=False):
    data_list = get_data(dataset_name, batch_size, results_path, debug)
    all_models, run_names, model_sizes = load_model_from_path(
        dataset_name, results_path
    )  # default list of models in load_models.py
    return data_list, all_models, run_names, model_sizes

In [ ]:
import yaml
from cyto_dl.models.utils.mlflow import get_config, load_model_from_checkpoint
from hydra._internal.utils import _locate
from hydra.utils import instantiate

from br.models.utils import get_all_configs_per_dataset


def load_model_from_mlflow(dataset, results_path, split="val"):
    TRACKING_URI = "https://mlflow.a100.int.allencell.org"
    MODEL_INFO = get_all_configs_per_dataset(results_path)
    models = MODEL_INFO[dataset]
    model_sizes = []
    all_models = []
    for i in models["run_ids"]:
        all_models.append(
            load_model_from_checkpoint(
                TRACKING_URI,
                i,
                path=f"checkpoints/{split}/loss/best.ckpt",
                strict=False,
            )
        )
        config = get_config(TRACKING_URI, i, "./tmp")
        model_sizes.append(config["model/params/total"])

    return all_models, models["names"], model_sizes

In [ ]:
TRACKING_URI = "https://mlflow.a100.int.allencell.org"
split = "train"
model = load_model_from_checkpoint(
    TRACKING_URI,
    "af07517eec5e496994ddc9d471eebb29",
    path=f"checkpoints/{split}/loss/best.ckpt",
    strict=False,
)

In [ ]:
model

In [ ]:
image_vae.

In [ ]:
from hydra._internal.utils import _locate
from hydra.utils import instantiate

MODEL_INFO = get_all_configs_per_dataset(results_path)
models = MODEL_INFO[dataset_name]
model_sizes = []
all_models = []
for ckpt_path in models["model_checkpoints"]:
    config_path = ckpt_path.split("ckpt")[0] + "yaml"
    with open(config_path) as stream:
        config = yaml.safe_load(stream)
    model_conf = config["model"]
    model_class = model_conf.pop("_target_")
    model_conf = instantiate(model_conf)
    model_class = _locate(model_class)
    all_models.append(
        model_class.load_from_checkpoint(ckpt_path, **model_conf, strict=True).eval()
    )
    model_sizes.append(config["model/params/total"])

In [ ]:
ckpt_path.split("ckpt")

In [ ]:
dataset_name = "npm1"
batch_size = 2
debug = True
results_path = (
    "/allen/aics/modeling/ritvik/projects/benchmarking_representations/br/configs/results/"
)
data_list, all_models, run_names, model_sizes = get_data_and_models(
    dataset_name, batch_size, results_path, debug
)

In [ ]:
import torch
from src.models.save_embeddings import save_embeddings

debug = False
# path = "./test5"
# path = "./npm1_test"
path = "./npm1_global"

params3_4 = {
    "eval_scaled_img_model_type": None,
    "eval_scaled_img_resolution": 35,
    "gt_mesh_dir": "/allen/aics/modeling/ritvik/projects/data/cellpack_npm1_spheres/",
    "gt_sampled_pts_dir": ".",
    "eval_scaled_img_model_type": "seg",
    "gt_scale_factor_dict_path": None,
    "mesh_ext": "ply",
}
# save_embeddings(path, data_list, all_models, run_names, debug, ['train', 'val', 'test'], device, None,
#                  [torch.nn.MSELoss(reduction='none')]*1, [False] * 1, None, [True]*1, [params3_4]
# )
save_embeddings(
    path,
    data_list,
    all_models,
    run_names,
    debug,
    ["train", "val", "test"],
    device,
    None,
    [None] * 1,
    [False] * 1,
    None,
    [False] * 1,
    [params3_4],
)

In [ ]:
# dataset_name = 'npm1_test'
run_names = ["equiv_vnn"]
from src.models.compute_features import get_embeddings

all_ret, df = get_embeddings(run_names, dataset_name)

In [ ]:
import pandas as pd

df = pd.read_csv("/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/var_npm1_manifest.csv")
cols_to_use = df.columns.difference(all_ret.columns).tolist() + ["CellId"]
all_ret = all_ret.merge(df[cols_to_use], on="CellId")
mesh_df = pd.read_csv(
    "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/data/var_npm1_manifest.csv"
)

In [ ]:
updated_feat_df = pd.read_csv(
    "/allen/aics/assay-dev/users/Alex/replearn/rep_paper/processing_data/npm1_fullres_features.csv"
)

In [ ]:
all_vals = []
for ind, row in updated_feat_df.iterrows():
    if row["connectivity_cc"] >= 5.0:
        new_val = ">=5"
    else:
        new_val = str(row["connectivity_cc"])
    all_vals.append(new_val)

In [ ]:
updated_feat_df["new_connectivity_thresh"] = all_vals

In [ ]:
updated_feat_df

In [ ]:
all_ret = all_ret.merge(updated_feat_df[["CellId", "new_connectivity_thresh"]], on="CellId")

In [ ]:
all_ret["new_connectivity_thresh"].value_counts()

In [ ]:
from src.features.classification import get_classification_df

connect_class = get_classification_df(all_ret, "new_connectivity_thresh")

In [ ]:
# regress_cols = ['avg_dists', 'std_dists', 'mean_volume',
#                                            'std_volume', 'mean_surface_area', 'std_surface_area']
cols = ["avg_dists"]
all_ret = all_ret.drop(columns=cols)

In [ ]:
all_ret = all_ret.merge(updated_feat_df[["CellId"] + regress_cols], on="CellId")

In [ ]:
from src.features.regression import get_regression_df

regress = get_regression_df(all_ret, regress_cols, None)

In [ ]:
regress.groupby(["target"]).mean()

In [ ]:
# regress.to_csv('./npm1_global/regression.csv')

In [ ]:
import pyvista as pv
from cyto_dl.image.transforms import RotationMask
from skimage.io import imread
from sklearn.decomposition import PCA
from src.data.utils import mesh_seg_model_output
from tqdm import tqdm

num_pieces = ">=5"

for num_pieces in ["1.0", "2.0", "3.0", "4.0", ">=5"]:
    # num_pieces = '4.0'
    this_sub_m = all_ret.loc[all_ret["new_connectivity_thresh"] == num_pieces].reset_index(
        drop=True
    )
    all_features = this_sub_m[[i for i in this_sub_m.columns if "mu" in i]].values
    latent_dim = 512
    dim_size = latent_dim
    x_label = "pcloud"
    pca = PCA(n_components=dim_size)
    pca_features = pca.fit_transform(all_features)
    pca_std_list = pca_features.std(axis=0)
    rank = 0
    all_xhat = []
    all_closest_real = []
    all_closest_img = []
    latent_walk_range = [-2, 0, 2]
    # latent_walk_range = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]
    for value_index, value in enumerate(tqdm(latent_walk_range, total=len(latent_walk_range))):
        z_inf = torch.zeros(1, dim_size)
        z_inf[:, rank] += value * pca_std_list[rank]
        z_inf = pca.inverse_transform(z_inf)

        dist = (all_features - z_inf) ** 2
        dist = np.sum(dist, axis=1)
        closest_idx = np.argmin(dist)
        closest_real_id = this_sub_m.iloc[closest_idx]["CellId"]
        mesh = pv.read(
            mesh_df.loc[mesh_df["CellId"] == closest_real_id]["mesh_path_noalign"].iloc[0]
        )
        mesh.save(f"./npm1_global/closest2/{num_pieces}_{rank}_{value_index}.ply")

In [ ]:
this_ret = all_ret
matrix = this_ret[[i for i in this_ret.columns if "mu" in i]].values

In [ ]:
from src.features.archetype import AA_Fast

n_archetypes = 5
aa = AA_Fast(n_archetypes, max_iter=1000, tol=1e-6).fit(matrix)

In [ ]:
import pandas as pd

archetypes_df = pd.DataFrame(aa.Z, columns=[f"mu_{i}" for i in range(matrix.shape[1])])

In [ ]:
archetypes_df

In [ ]:
all_features = matrix
for i in range(n_archetypes):
    this_mu = archetypes_df.iloc[i].values
    dist = (all_features - this_mu) ** 2
    dist = np.sum(dist, axis=1)
    closest_idx = np.argmin(dist)
    closest_real_id = this_ret.iloc[closest_idx]["CellId"]
    print(dist, closest_real_id)
    mesh = pv.read(mesh_df.loc[mesh_df["CellId"] == closest_real_id]["mesh_path_noalign"].iloc[0])
    mesh.save(f"./npm1_global/archetype2/{i}.ply")

In [ ]:
# all_features =  matrix
# for i in range(n_archetypes):
#     this_mu = archetypes_df.iloc[i].values
#     dist = (all_features - this_mu) ** 2
#     dist = np.sum(dist, axis=1)
#     closest_idx = np.argmin(dist)
#     closest_real_id = this_ret.iloc[closest_idx]['CellId']
#     print(dist, closest_real_id)
#     mesh = pv.read(mesh_df.loc[mesh_df['CellId'] == closest_real_id]['mesh_path_noalign'].iloc[0])
#     mesh.save(f'./npm1_test/closest/archetype/{i}.ply')

In [ ]:
all_ret["STR_connectivity_cc_thresh"]

In [ ]:
for hh in all_ret["new_connectivity_thresh"].unique():
    this_ret = all_ret.loc[all_ret["new_connectivity_thresh"] == hh].reset_index(drop=True)
    labels = this_ret["structure_name"].values
    # labels = this_ret['cell_stage_fine'].values
    matrix = this_ret[[i for i in this_ret.columns if "mu" in i]].values
    from src.features.archetype import AA_Fast

    n_archetypes = 3
    aa = AA_Fast(n_archetypes, max_iter=1000, tol=1e-6).fit(matrix)
    all_features = matrix
    for i in range(n_archetypes):
        print(hh, i)
        this_mu = archetypes_df.iloc[i].values
        dist = (all_features - this_mu) ** 2
        dist = np.sum(dist, axis=1)
        closest_idx = np.argmin(dist)
        closest_real_id = this_ret.iloc[closest_idx]["CellId"]
        print(dist, closest_real_id)
        mesh = pv.read(
            mesh_df.loc[mesh_df["CellId"] == closest_real_id]["mesh_path_noalign"].iloc[0]
        )
        mesh.save(f"./npm1_global/archetype/per_piece/{hh}_{i}_{closest_real_id}.ply")

In [ ]:
from skimage.io import imread

img = imread(all_ret.loc[all_ret["CellId"] == 974872]["crop_seg_masked"].iloc[0])

In [ ]:
all_ret["volume_of_nucleus_um3"] = all_ret["NUC_shape_volume_lcc"] * 0.108**3

In [ ]:
bins = [(370.839, 577.444], (577.444, 784.05], (163.2, 370.839], (784.05, 990.656],  ]

In [ ]:
all_ret["volume_of_nucleus_um3"].max()

In [ ]:
all_ret["outlier"].value_counts()

In [ ]:
[i for i in all_ret.columns if "outlier" in i]

In [ ]:
feat = "volume_of_nucleus_um3"
upper = np.quantile(all_ret[feat], q=0.975)
lower = np.quantile(all_ret[feat], q=0.025)

this = all_ret.loc[all_ret[feat] < upper]
this = this.loc[this[feat] > lower].reset_index(drop=True)
# this = all_ret

# this = this.loc[this['CellId'] != 956566].reset_index(drop=True)

In [ ]:
this[feat].hist()

In [ ]:
this["vol_bins"] = pd.cut(this[feat], bins=5)
this["vol_bins_ind"] = pd.factorize(this["vol_bins"])[0]

# this['vol_bins'] = pd.qcut(this[feat], q=5)
# this['vol_bins_ind'] = pd.factorize(this['vol_bins'])[0]

In [ ]:
this["vol_bins"].value_counts()

In [ ]:
this["vol_bins_ind"].value_counts()

In [ ]:
this["vol_bins_ind"].value_counts()

In [ ]:
this["vol_bins"].astype(str).iloc[0]

In [ ]:
all_features = this[[i for i in this.columns if "mu" in i]].values
this["vol_bins"] = this["vol_bins"].astype(str)
for hh in this["vol_bins"].unique():
    this_ret = this.loc[this["vol_bins"] == hh].reset_index(drop=True)

    this_mu = np.expand_dims(
        this_ret[[i for i in this_ret.columns if "mu" in i]].mean(axis=0), axis=0
    )
    dist = (all_features - this_mu) ** 2
    # dist = np.sum(dist, axis=1)
    k = 2
    # print(min(latent_dim, all_features.shape[0]))
    inds = np.argpartition(dist.sum(axis=-1), k)[:k]  # get 10 closest
    closest_samples = this.iloc[inds].reset_index(drop=True)
    for ind, row in closest_samples.iterrows():
        # closest_real_id = this.iloc[closest_idx]['CellId']
        closest_real_id = row["CellId"]
        print(
            closest_idx,
            this_ret["vol_bins"].unique(),
            all_features.shape,
            this_ret.shape,
            this_ret["NUC_shape_volume_lcc"].mean(),
            closest_real_id,
        )
        mesh = pv.read(
            mesh_df.loc[mesh_df["CellId"] == closest_real_id]["mesh_path_noalign"].iloc[0]
        )
        mesh.save(f"./npm1_global/vol_bin2/{hh}_{ind}_{closest_real_id}.ply")

In [ ]:
dist.shape

In [ ]:
this_mu.shape

In [ ]:
this